# Fase 2 Experimentos


## Datasets experimentos


In [1]:
global datasets, metodos
from ipynb.fs.full.datasets import *
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
#Diccionario con los datasets de los experimentos
datasets = {"mnist_numbers" : mnist_numbers(), "mnist_with_clothes" : mnist_with_clothes(), "chinese_mnist" : chinese_mnist(),   \
            "wine" : wine(), "gamma_telescope" : gamma_telescope(),   \
            "image_segmentation": image_segmentation(), "digits_dataset": digits_dataset(), "breast_cancer": load_breast_cancer()}


# Algoritmos de clasificación utilizados durante los experimentos de la libreria sklearn
metodos = {"SGDClassifier" : SGDClassifier(), "gaussianNB": GaussianNB(), "random_forest": RandomForestClassifier(),  \
           "logistic_regression": LogisticRegression(),"xgboost": GradientBoostingClassifier(), "mlp_classifier" : MLPClassifier()}

C:\Users\Galo\Documents\Matematicas\4 Mates\TFG\datasets.ipynb:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "    return df"


## Etiquetas fuzzy

In [11]:
def fuzzy_labels(X, y, metodo, num_veces, n, path, logger):
    import csv
    from sklearn.model_selection import StratifiedKFold,train_test_split
    from sklearn.linear_model import SGDClassifier
    import numpy as np
    import pandas as pd
    """
    Función que obtiene etiquetas fuzzy para un metodo y dataset
    """
    clases = np.unique(y)
    n_splits = 5
    for iter in range(n):    
        
        val_predict = np.zeros((num_veces,len(y)))
        
        for i in range(num_veces):
            logger.info(f"Iteracion{iter}")
            logger.info(f"Partición {i} del dataset")
            
            particiones = StratifiedKFold(n_splits=n_splits, shuffle=True)     
            
            for j, (train_index, test_index) in enumerate(particiones.split(X, y)):

                logger.debug(f"Particion test {j}")
                
                #Conjunto de datos de entrenamiento sin filtrar
                X_train = X[train_index,]
                y_train = y[train_index]
    
                #Datos del test
                X_test = X[test_index,]
                y_test = y[test_index]
                
                # generar modelo y predecir con los datos filtrados
                modelo = metodo.fit(X_train,y_train)         
                y_pred = modelo.predict(X_test) #  predecir solo el test
                val_predict[i,test_index] = y_pred
        if iter == 0:
            data_probs = calculate_proportions(val_predict)
        else:
            probs = calculate_proportions(val_predict)
            data_probs = change_probs(data_probs,probs,clases)
    logger.info("Pasamos los datos a un csv")
    df = normal(data_probs,clases,n)
    df.to_csv(path, index=False)
    return df

def change_probs(data_probs,probs,clases):
    for i in range(len(data_probs)):
        for clase in clases:
            if clase in probs[i]:
                if clase in data_probs[i]:
                    data_probs[i][clase] =  data_probs[i][clase]+probs[i][clase]
                else:
                    data_probs[i][clase] =  probs[i][clase]
    return data_probs
        
def calculate_proportions(matrix):
    import numpy as np
    proportions = []
    for row in matrix.T:
        unique, counts = np.unique(row, return_counts=True)
        total_elements = row.size
        row_proportions = {val: [count / total_elements] for val, count in zip(unique, counts)}
        proportions.append(row_proportions)
    return proportions   

def normal(data_probs,clases,n):
    
    import pandas as pd 
    import numpy as np
    
    df = pd.DataFrame(columns=clases)
    dist = {clase: [] for clase in clases}
    for data in data_probs:
        for clase in clases:
            if clase in data: 
                padded = data[clase] + (n-len(data[clase]))*[0]
                normal = [np.mean(padded),np.var(padded,ddof=1)]
            else:
                normal = [0,0]
            dist[clase].append(normal)
    df = pd.DataFrame(dist) 
    return df
        

In [15]:
import numpy as np
from sklearn.model_selection import StratifiedKFold,train_test_split
X = np.array([[1,2],[3,4]])
y = np.array([0,1])
print(val_predict[:,[2,1]])
calculate_proportions(val_predict)

IndexError: index 2 is out of bounds for axis 1 with size 2

In [21]:
particiones = StratifiedKFold(n_splits=2, shuffle=True)
X = np.random.randint(0, 10, size=(4, 4))
y = np.array([0,1,0,1])
print(X)
for j, (train_index, test_index) in enumerate(particiones.split(X, y)):
    print(test_index)
    print(X[1,test_index])

[[2 6 8 8]
 [0 7 7 6]
 [9 6 5 6]
 [9 5 6 3]]
[2 3]
[7 6]
[0 1]
[0 7]


## Generates a dictionary with the file names and the files id of the google drive folder

In [3]:
def get_drive_files_dict(folder_id):
    import requests
    from bs4 import BeautifulSoup
    import re
    """
    Given a public Google Drive folder ID, fetch the embedded folder view
    and create a dictionary mapping each file name to its file ID.
    """
    # URL for the embedded folder view
    folder_url = f"https://drive.google.com/embeddedfolderview?id={folder_id}"
    response = requests.get(folder_url)
    
    if response.status_code != 200:
        raise Exception("Failed to fetch the folder page. Make sure the folder is public.")
    
    soup = BeautifulSoup(response.content, 'html.parser')
    files_dict = {}
    
    # Iterate through all links; each should represent a file in the folder
    for link in soup.find_all('a'):
        file_name = link.text.strip()
        href = link.get('href')
        if not href:
            continue
        
        # The href should include the file id in the format: /file/d/FILE_ID/view
        match = re.search(r'/file/d/([a-zA-Z0-9_-]+)/', href)
        if match:
            file_id = match.group(1)
            files_dict[file_name] = file_id

    return files_dict

## Código donde se realiza cada experimento


In [4]:
def experimento(metodo_exp, parametros, df_str, now, logger):

    import numpy as np
    import os
    import csv
    from datetime import datetime
    import time 
    import json
    
    """
    Función que realiza un experimento estructurado de la siguiente manera para cada dataset->
    1. Introduce ruido en las etiquetas del dataset
    2. Aplica a todo el dataset el algoritmo de detección de ruido y obtiene una matriz de confusion
    3. Aplica el algoritmo de detección de ruido y obtiene intervalos de confianza para los kappas
    
    Parametros de entrada de la función->
    metodo_exp : metodo utilizado en el experimento
    parametros : hiperparametros elegidos para el experimento
    datasets_exp : lista con el nombre de los dataframes que se utilizaran en el experimento
    now : fecha y hora del comienzo de experimento para utilizarlo de nombre del archivo donde se guardaran los resultados
    """
    
    start_time = time.time()

    #Parámetros
    metodo = metodos.get(metodo_exp)
    ruido = parametros

    date_time = now.strftime("%m/%d/%Y")
    file_name = now.strftime("_%m_%d_%Y_%H_%M_%S")
    path = "normal_" + df_str + "_" + f"{ruido}" + "_" + metodo_exp + "_" + file_name + ".csv"

    df = datasets.get(df_str)
    
    y_column = "label"
    X_columns = [c for c in df.columns if c!=y_column]
    X = df[X_columns].to_numpy() #Atributos
    
    if ruido != 0:
        path2 = "ruido" + file_name + ".csv"
        folder_id = "15Rt7UEmWmCdeRH_v9JmBmfR7WPmzTyth"
        files_dict = get_drive_files_dict(folder_id)
        file_name = f"ruido_en_{df_str}_{ruido}.csv"
        file_id = files_dict.get(file_name)
        url = f"https://drive.google.com/uc?id={file_id}"  # Replace with actual file ID
        noise_df = pd.read_csv(url)
        y = noise_df["Etiquetas con ruido"].to_numpy()
    else:
        y = df[y_column].to_numpy() #Etiquetas

    logger.info("Dataset : " + df_str)

    num_veces = 10
    n = 20
    
    fuzzy_labels(X, y, metodo, num_veces, n, path, logger)
    
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time
    
    # Convert to hours, minutes, and seconds
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = int(elapsed_time % 60)

    tiempo = f"{hours}h_{minutes}m_{seconds}s"
    
    return path, date_time, tiempo

## Función para hacer los experimentos en periodos de desconexión

In [5]:
import pandas as pd
from datetime import datetime
from ipynb.fs.full.datasets import * 
import logging

def experimentos_no_realizados():
    
    """
    Función que parte de una base de experimentos en un excel llamado "experimentos.xlsx", y va realizando los experimentos no empezados
    de forma secuencial
    """
   
    experimentos = pd.read_excel("experimentos_fase2.xlsx")
    experimentos_no_hechos = experimentos[experimentos["FECHA"].isna()]
    # print(experimentos_no_hechos.head())

    # Create a logger
    logger = logging.getLogger("my_logger")
    logger.setLevel(logging.DEBUG)  # Set the logging level
    
    # Create a FileHandler to write logs to a file
    file_handler = logging.FileHandler("experimentos_fase2.log")
    file_handler.setLevel(logging.DEBUG)  # Set handler logging level
    
    # Create a log message format
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)  # Attach formatter to handler
    
    # Add handler to the logger
    logger.addHandler(file_handler)
    
    for index, row in experimentos_no_hechos.iterrows():

        #Dataset experimento
        datasets_exp = experimentos_no_hechos.at[index,"DATASETS"]

        # Parametros experimento
        num = experimentos_no_hechos.at[index,"PARAMETROS"]
        parametros = float(num)

        #Metodo experimento
        metodo = experimentos_no_hechos.at[index,"METODO"]

        #Empezamos el experimento apuntando en el archivo ".log" lo que vamos a realizar
        exp_str = f"Empezamos el experimento con el método " + metodo + " y parametros " + f"{num}"
        logger.critical(exp_str)

        #Tiempo actual
        now = datetime.now()
        
        # Realizamos el experimento
        file_name, date_time, time = experimento(metodo,parametros,datasets_exp,now,logger)

        # Actualizamos el dataframe con el registro de los experimentos
        experimentos.loc[index, "FECHA"] = date_time  
        experimentos.loc[index, "NOMBRE ARCHIVO"] = file_name
        experimentos.loc[index, "TIEMPO"] = time

        #Una vez finalizado el experimento, actualizamos el excel con los resultados obtenidos
        experimentos.to_excel("experimentos_fase2.xlsx", index=False)
    file_handler.close()
    logger.removeHandler(file_handler)
    print("Terminados todos los experimentos")
    return 1

In [6]:
experimentos_no_realizados()

KeyboardInterrupt: 

## Funcion para cerrar los handlers cuando surga un error en el código (no forma parte del programa si no hay fallos)

In [14]:
import logging

def close_all_handlers():
    """Closes and removes all handlers from all loggers."""
    for logger_name in logging.root.manager.loggerDict:
        logger = logging.getLogger(logger_name)
        if hasattr(logger, "handlers"):  # Check if the logger has handlers
            for handler in logger.handlers[:]:  # Iterate over a copy of the handlers list
                handler.close()
                logger.removeHandler(handler)

# Example usage
close_all_handlers()